# TODO

- Get all genres from selenium data &#9745;
- Get distribution of language reviews &#9745;
- Pandas dataframe creation
- Get all book comments
- Clean everything
- Automate the process
- Investigate about executing multiples requests concurrently

In [1]:
%pip install bs4 -q
%pip install selenium -q
%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [6]:
# Goodreads request
book_gr = f'https://www.goodreads.com/book/show/58473038'
soup_gr = BeautifulSoup(requests.get(book_gr).text, features='html.parser')
type(soup_gr)

bs4.BeautifulSoup

In [5]:
# Goodreads basic scraping
title = soup_gr.find(class_='Text Text__title1').text
authors = [cla.text for cla in soup_gr.find_all(class_='ContributorLink__name')]
rating = soup_gr.find(class_="RatingStatistics__rating").text
desc = soup_gr.find(class_="DetailsLayoutRightParagraph__widthConstrained").text
pub_info = soup_gr.find('p', {'data-testid': 'publicationInfo'}).text
cover = soup_gr.find(class_='ResponsiveImage').get('src')

book_data = {
    'title':title, 
    'authors':authors,
    'rating_value':rating,
    'desc':desc,
    'pub_info':pub_info,
    'cover':cover,
}

book_data

{'title': 'Teenage Mutant Ninja Turtles: The Last Ronin',
 'authors': ['Kevin Eastman', 'Peter Laird', 'Tom Waltz', 'Kevin Eastman'],
 'rating_value': '4.41',
 'desc': 'Who is the Last Ronin? In a future, battle-ravaged New York City, a lone surviving Turtle embarks on a seemingly hopeless mission seeking justice for the family he lost. From legendary TMNT co-creators Kevin Eastman and Peter Laird, get ready for the final story of the Teenage Mutant Ninja Turtles three decades in the making!What terrible events destroyed his family and left New York a crumbling, post-apocalyptic nightmare? All will be revealed in this climactic Turtle tale that sees longtime friends becoming enemies and new allies emerging in the most unexpected places. Can the surviving Turtle triumph?Eastman and Laird are joined by writer Tom Waltz, who penned the first 100 issues of IDW’s ongoing TMNT series, and artists Esau & Isaac Escorza (Heavy Metal) and Ben Bishop (The Far Side of the Moon) with an Introductio

In [14]:
from selenium import webdriver
from datetime import datetime

browser = webdriver.Firefox()

url = 'https://www.goodreads.com/book/show/58473038'
browser.get(url)
html_source = browser.page_source
browser.quit()

data_str = re.findall('"details":.*,"work"', html_source)[0].replace('"details":', '').replace(',"work"', '')

data_dict = json.loads(data_str)

book_data['id'] = url.split('/')[-1]
book_data['format'] = data_dict['format']
book_data['num_pages'] = data_dict['numPages']
book_data['publication_timestamp'] = data_dict['publicationTime']
book_data['publication_date'] = datetime.fromtimestamp(int(data_dict['publicationTime'])/1000).strftime("%Y-%m-%d")
book_data['publisher'] = data_dict['publisher']
book_data['isbn'] = data_dict['isbn']
book_data['isbn13'] = data_dict['isbn13']
book_data['language'] = data_dict['language']['name']

rating_count = re.findall('"ratingCount":\d+', html_source)[0].replace('"ratingCount":', '')
review_count = re.findall('"reviewCount":\d+', html_source)[0].replace('"reviewCount":', '')

# Reviews count by language  
count_lang = re.findall('"count":\d+,"isoLanguageCode":"[a-z]+"', html_source)
dict_count_lang = [json.loads('{' + lang + '}') for lang in count_lang]
book_data['review_count_by_lang'] = {item['isoLanguageCode']:item['count'] for item in dict_count_lang}

# Genres 
genres = re.findall('"bookGenres":.*}}],"details":', html_source)[0].replace(',"details":', '')
dict_genres = [json.loads('{' + genres + '}')]
book_data['genres'] = [genre['genre']['name'] for genre in dict_genres[0]['bookGenres']]

book_data['rating_count'] = rating_count
book_data['review_count'] = review_count

book_data

{'title': 'Teenage Mutant Ninja Turtles: The Last Ronin',
 'authors': ['Kevin Eastman', 'Peter Laird', 'Tom Waltz', 'Kevin Eastman'],
 'rating_value': '4.41',
 'desc': 'Who is the Last Ronin? In a future, battle-ravaged New York City, a lone surviving Turtle embarks on a seemingly hopeless mission seeking justice for the family he lost. From legendary TMNT co-creators Kevin Eastman and Peter Laird, get ready for the final story of the Teenage Mutant Ninja Turtles three decades in the making!What terrible events destroyed his family and left New York a crumbling, post-apocalyptic nightmare? All will be revealed in this climactic Turtle tale that sees longtime friends becoming enemies and new allies emerging in the most unexpected places. Can the surviving Turtle triumph?Eastman and Laird are joined by writer Tom Waltz, who penned the first 100 issues of IDW’s ongoing TMNT series, and artists Esau & Isaac Escorza (Heavy Metal) and Ben Bishop (The Far Side of the Moon) with an Introductio

In [13]:
dict_genres

{'bookGenres': [{'__typename': 'BookGenre',
   'genre': {'__typename': 'Genre',
    'name': 'Comics',
    'webUrl': 'https://www.goodreads.com/genres/comics'}},
  {'__typename': 'BookGenre',
   'genre': {'__typename': 'Genre',
    'name': 'Graphic Novels',
    'webUrl': 'https://www.goodreads.com/genres/graphic-novels'}},
  {'__typename': 'BookGenre',
   'genre': {'__typename': 'Genre',
    'name': 'Fiction',
    'webUrl': 'https://www.goodreads.com/genres/fiction'}},
  {'__typename': 'BookGenre',
   'genre': {'__typename': 'Genre',
    'name': 'Fantasy',
    'webUrl': 'https://www.goodreads.com/genres/fantasy'}},
  {'__typename': 'BookGenre',
   'genre': {'__typename': 'Genre',
    'name': 'Graphic Novels Comics',
    'webUrl': 'https://www.goodreads.com/genres/graphic-novels-comics'}},
  {'__typename': 'BookGenre',
   'genre': {'__typename': 'Genre',
    'name': 'Science Fiction',
    'webUrl': 'https://www.goodreads.com/genres/science-fiction'}},
  {'__typename': 'BookGenre',
   'ge

In [97]:
# Pandas dataframe generation

import pandas as pd

df = pd.DataFrame([book_data])
df

,title,authors,rating_value,desc,pub_info,cover,id,format,num_pages,publication_timestamp,publication_date,publisher,isbn,isbn13,language,review_count_by_lang,genres,rating_count,review_count
0,Teenage Mutant Ninja Turtles: The Last Ronin,"[Kevin Eastman, Peter Laird, Tom Waltz, Kevin ...",4.41,"Who is the Last Ronin? In a future, battle-rav...","First published July 5, 2022",https://images-na.ssl-images-amazon.com/images...,58473038,Hardcover,224,1657004400000,2022-07-05,IDW Publishing,1684058414,9781684058419,English,"{'en': 782, 'cs': 4, 'es': 22, 'it': 4, 'sr': ...","[Comics, Graphic Novels, Fiction, Fantasy, Gra...",8602,861
